In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
loader = WebBaseLoader("https://www.wikihow.com/Do-Yoga")

docs = loader.load()


In [3]:
text_splitter = RecursiveCharacterTextSplitter()

documents = text_splitter.split_documents(docs)

llm = ChatOpenAI()

embeddings = OpenAIEmbeddings()

In [4]:
vector = FAISS.from_documents(documents, embeddings)

retriever = vector.as_retriever()

In [5]:
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])

In [6]:
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

sample_answer = """Some key points for Yoga begginners are:
    1. Find a comfortable place and time to practice.
    2. Set a routine that suits you.
    and so on..
    ."""

chat_history = [HumanMessage(content="What are the key things to consider for someone starting to practice Yoga?"), 
                AIMessage(content=sample_answer)]


prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])

document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

output = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Can you elaborate on the first point?"
})

print(output["answer"])

Of course! When starting to practice yoga, it's important to find a comfortable and quiet place where you can focus and move freely. Make sure the space is free from distractions like TVs, radios, or phones. Ensure you have enough room to move and stretch without bumping into furniture or objects. Consider practicing in a room with good ventilation and lighting. Creating a calming environment can enhance your yoga experience and help you relax and focus during your practice.
